In [ ]:
!apt-get update
!apt-get install -y libportaudio2 portaudio19-dev


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 261 kB in 2s (153 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem

In [ ]:
!pip install sounddevice
import sounddevice as sd

In [ ]:
!pip install SpeechRecognition vosk whisper
!pip install pyaudio

  Using cached whisper-1.1.10-py3-none-any.whl


In [ ]:
from google.colab import files

# This will open a file upload dialog in your browser.
uploaded = files.upload()

Saving lab3sample.wav to lab3sample (1).wav


In [ ]:
# Step 1: Uninstall any incorrect or conflicting versions to ensure a clean state.
# The '-y' flag automatically confirms the uninstall.
!pip uninstall whisper openai-whisper -y

# Step 2: Install the correct library from OpenAI.
!pip install openai-whisper

# Step 3: Install ffmpeg, a command-line tool that Whisper requires for audio processing.
# This is a necessary step in Google Colab.
!sudo apt-get -q install ffmpeg

Found existing installation: whisper 1.1.10
Uninstalling whisper-1.1.10:
  Successfully uninstalled whisper-1.1.10
Found existing installation: openai-whisper 20250625
Uninstalling openai-whisper-20250625:
  Successfully uninstalled openai-whisper-20250625
  Using cached openai_whisper-20250625-py3-none-any.whl
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [ ]:
import speech_recognition as sr
import os
from vosk import Model, KaldiRecognizer
import wave
import json

def recognize_speech_from_file(file_path):
    """
    Recognizes speech from an audio file using Google Speech Recognition, Vosk, and Whisper.

    Args:
        file_path (str): The path to the audio file.

    Returns:
        dict: A dictionary containing the recognized text from each method.
    """
    recognizer = sr.Recognizer()
    results = {}

    print("Speak something...")

    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)

    # --- Google Speech Recognition (Online) ---
    print("Recognizing using Google Speech API...")
    try:
        google_text = recognizer.recognize_google(audio_data)
        results["Google API Output"] = google_text
        print("Speech successfully converted to text!")
    except sr.UnknownValueError:
        results["Google API Output"] = "Speech Recognition could not understand audio. Please try speaking more clearly."
    except sr.RequestError as e:
        results["Google API Output"] = f"Could not request results from Google Speech Recognition service; {e}"

    # --- Vosk (Offline) ---
    print("\nRecognizing using Vosk...")
    try:
        model = Model("vosk-model-small-en-us-0.15") # Make sure you have downloaded and placed the model in the correct path
        wf = wave.open(file_path, "rb")
        rec = KaldiRecognizer(model, wf.getframerate())
        rec.SetWords(True)

        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            if rec.AcceptWaveform(data):
                pass

        vosk_result = json.loads(rec.FinalResult())
        results["Vosk Output"] = vosk_result['text']
        print("Speech successfully converted to text!")

    except Exception as e:
        results["Vosk Output"] = f"Error with Vosk: {e}"

    # --- Whisper (Offline) ---
    print("\nRecognizing using Whisper...")
    try:
        whisper_text = recognizer.recognize_whisper(audio_data)
        results["Whisper Output"] = whisper_text
        print("Speech successfully converted to text!")
    except sr.UnknownValueError:
        results["Whisper Output"] = "Whisper could not understand audio."
    except sr.RequestError as e:
        results["Whisper Output"] = f"Could not request results from Whisper; {e}"


    return results

if __name__ == "__main__":
    # Replace with the path to your audio file from Google Drive
    audio_file = "lab3sample.wav"

    if os.path.exists(audio_file):
        recognized_texts = recognize_speech_from_file(audio_file)
        print("\n--- Recognition Results ---")
        for method, text in recognized_texts.items():
            print(f"{method}: '{text}'")
    else:
        print(f"Error: The file '{audio_file}' was not found.")
        print("Please download the 'lab3sample.wav' file and place it in the same directory as the script.")

Speak something...
Recognizing using Google Speech API...
Speech successfully converted to text!

Recognizing using Vosk...

Recognizing using Whisper...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 107MiB/s]


Speech successfully converted to text!

--- Recognition Results ---
Google API Output: 'I believe you are just talking nonsense'
Vosk Output: 'Error with Vosk: Failed to create a model'
Whisper Output: ' I believe you're just talking nonsense.'
